<a href="https://colab.research.google.com/github/PearceStrickland/Momentum-Trading-Algo/blob/main/Momentum_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/backtrader2/backtrader

  Cloning https://github.com/backtrader2/backtrader to /tmp/pip-req-build-ml0b_uaf
  Running command git clone -q https://github.com/backtrader2/backtrader /tmp/pip-req-build-ml0b_uaf
  Created wheel for backtrader: filename=backtrader-1.9.76.123-py3-none-any.whl size=419488 sha256=961059e75442ff5a0455278f21026fef29d236616a8ed958177bf8363f06dd3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-bvgbhtnz/wheels/c5/d9/ed/07ba378f0314939bc1697e304d2c3d45fb9a9c09f3da94b2a3
Successfully built backtrader


In [ ]:
pip install git+https://github.com/quantopian/pyfolio

  Cloning https://github.com/quantopian/pyfolio to /tmp/pip-req-build-6ygk2b_c
  Running command git clone -q https://github.com/quantopian/pyfolio /tmp/pip-req-build-6ygk2b_c
     |████████████████████████████████| 52 kB 804 kB/s 
  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=75775 sha256=1e56fa1e7f1073225f99affacffc101e73dd097192d789358d30a4db143112c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-doy8956w/wheels/2d/aa/24/c99ed55ef37c69e33815248c1622cdb81b65ec753868004c28
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39777 sha256=4341a945d66e494be241ed21b9c9342df8d5871bf73b241192ade75294ecbf71
  Stored in directory: /root/.cache/pip/wheels/d9/91/4b/654fcff57477efcf149eaca236da2fce991526cbab431bf312
Successfully built pyfolio empyrical


In [ ]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 6.8 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=4d22dcad35887920464b6f9dae092f91dda5237e9c3ebc7cbea46cec8b0c26f5
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import backtrader as bt
import pandas as pd
import pyfolio as pf
from datetime import datetime
import matplotlib
matplotlib.use('Agg')
from google.colab import files
import yfinance as yf

MACD cross strategy using ATR for stop loss


In [ ]:
class MACDCross(bt.Strategy):
  params = (
      # Standard MACD parameters
      #12 period EMA
      ('macd1', 12),
      #26 period EMA
      ('macd2', 26),
      #9 period signal line
      ('macdsig', 9),
      #A calculated using 14 periods
      ('atrperiod', 14),
      #Amount of true range distance you want for stop loss. Higher number=lower stop loss
      ('atrdist', 2.0), 

  ) 
  def log(self, txt, dt=None):
      ''' 
      Summary: This function allows us to print out whats going on inside the strategy
      Parameters: 
        - txt: Text to print out
        - dt: date that strategy event occured
      '''
      dt = dt or self.datas[0].datetime.date(0)
      print('%s, %s' % (dt.isoformat(), txt))
  
  def __init__(self):
    self.dataclose=self.datas[0].close
    self.order = None
    self.buyprice = None
    self.buycomm = None
    self.rsi = bt.indicators.RSI(self.datas[0])
    self.macd = bt.indicators.MACD(self.data,
                                       period_me1=self.p.macd1,
                                       period_me2=self.p.macd2,
                                       period_signal=self.p.macdsig)
    #Crossover indicatpr if data 1 inputted croses over data 2 value signal is 1 and if data 2 crosses data 1 value is negative 1
    self.mcross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)
    #Stoploss indicator using ATR
    self.atr = bt.indicators.ATR(self.data, period=self.p.atrperiod)
  
  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
            return
    if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)
    # After order has been either accepted or rejected, set order to none
    self.order = None

  def notify_trade(self, trade):
        """
        Summary: Another fancy print statement, except this one will print out 
        the P/L of a complete trade 
        """
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


  def next(self):
    if self.order:
            return
    if not self.position:  # not in the market
      if self.mcross[0] > 0.0:
        self.order = self.buy()
        stoprange = self.atr[0] * self.p.atrdist
        self.stopamount = self.data.close[0] - stoprange
    else:
      pclose = self.data.close[0]
      stopamount = self.stopamount
      if pclose < stopamount:
        self.close()  # stop met - get out
      else:
        pdist = self.atr[0] * self.p.atrdist
        # Update only if greater than
        self.stopamount = max(stopamount, pclose - pdist)
    




Broker Code

In [ ]:
# CREATE SIMPLE BROKER AND ADD STRATEGY

# create a "Cerebro" engine instance, Cerebro is what simulates a broker
cerebro = bt.Cerebro()

# Set Strategy Paramerters
pv = 10000
stock="SPY"

# Create a data feed
data = bt.feeds.YahooFinanceData(dataname=stock,
                                 fromdate=datetime(1995, 8, 9),
                                 todate=datetime(2021, 10, 31))

cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(MACDCross)  # Add the trading strategy

cerebro.broker.setcash(pv) # Set our desired cash start

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission
cerebro.broker.setcommission(commission=0.00)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % pv)

# These lines execute your code, and run Pyfolio on your strategy
# Pyfolio gives you a bunch of metrics to assess your strategy performance
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
strat=cerebro.run()

#Plotting , uncomment to download nice chart of trades and strategy performance
cerebro.plot()[0][0].savefig('smaStrat.png', dpi=300)
#files.download('smaStrat.png')

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

pyfoliozer = strat[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
simple_tear_sheet = pf.create_simple_tear_sheet(returns)

ts = pf.create_returns_tear_sheet(returns, return_fig=True)
ts.savefig('pyfolio_returns_tear_sheet.png')



Starting Portfolio Value: 10000.00
1995-11-06, BUY EXECUTED, Price: 37.09, Cost: 370.90, Comm 0.00
1995-12-19, SELL EXECUTED, Price: 38.23, Cost: 370.90, Comm 0.00
1995-12-19, OPERATION PROFIT, GROSS 11.40, NET 11.40
1996-01-24, BUY EXECUTED, Price: 38.77, Cost: 387.70, Comm 0.00
1996-02-20, SELL EXECUTED, Price: 40.61, Cost: 387.70, Comm 0.00
1996-02-20, OPERATION PROFIT, GROSS 18.40, NET 18.40
1996-03-21, BUY EXECUTED, Price: 41.28, Cost: 412.80, Comm 0.00
1996-04-11, SELL EXECUTED, Price: 39.98, Cost: 412.80, Comm 0.00
1996-04-11, OPERATION PROFIT, GROSS -13.00, NET -13.00
1996-04-23, BUY EXECUTED, Price: 41.10, Cost: 411.00, Comm 0.00
1996-05-08, SELL EXECUTED, Price: 40.30, Cost: 411.00, Comm 0.00
1996-05-08, OPERATION PROFIT, GROSS -8.00, NET -8.00
1996-05-14, BUY EXECUTED, Price: 42.17, Cost: 421.70, Comm 0.00
1996-06-19, SELL EXECUTED, Price: 42.19, Cost: 421.70, Comm 0.00
1996-06-19, OPERATION PROFIT, GROSS 0.20, NET 0.20
1996-07-02, BUY EXECUTED, Price: 43.03, Cost: 430.30, C

Start date,1995-08-09
End date,2021-10-29
Total months,314
,Backtest
Annual return,0.928%
Cumulative returns,27.403%
Annual volatility,1.582%
Sharpe ratio,0.59
Calmar ratio,0.20
Stability,0.52
Max drawdown,-4.652%


<IPython.core.display.Javascript object>

Start date,1995-08-09
End date,2021-10-29
Total months,314
,Backtest
Annual return,0.928%
Cumulative returns,27.403%
Annual volatility,1.582%
Sharpe ratio,0.59
Calmar ratio,0.20
Stability,0.52
Max drawdown,-4.652%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,4.65,1999-12-31,2002-08-05,2013-04-10,3464
1,3.55,2014-12-05,2016-02-09,2017-10-03,738
2,2.09,2018-08-29,2019-01-03,2019-01-31,112
3,1.95,2018-01-26,2018-05-03,2018-08-29,154
4,1.85,2020-06-08,2020-06-11,2020-08-05,43


<IPython.core.display.Javascript object>